##convert xls to xlsx

In [2]:
from xls2xlsx import XLS2XLSX
x2x = XLS2XLSX("feb25.xls")
x2x.to_xlsx("feb25.xlsx")

'feb25.xlsx'

##read the excel file   

In [26]:
import pandas as pd

df = pd.read_excel('D:\\codes\\bank statement extractor\\apr25.xls', engine='xlrd')

print(df.columns)




Index(['HDFC BANK Ltd.                                      Page No .:   1                                          Statement of accounts',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6'],
      dtype='object')


In [23]:
print(df.to_string())

                                                                                                                                             Date                                                                                                Narration      Chq./Ref.No.   Value Dt          Withdrawal Amt.        Deposit Amt.     Closing Balance
0                                                                                                                                        01/02/25          UPI-ICCL  MUTUAL FUNDS  -MFAUTOPAY.ELEMENTS@HDFCBANK-HDFC0000060-503214109548-ONLY RS 5000 WILL  0000503214109548   01/02/25                     5000                 NaN            54363.03
1                                                                                                                                        01/02/25          UPI-ICCL  MUTUAL FUNDS  -MFAUTOPAY.ELEMENTS@HDFCBANK-HDFC0000060-503214113379-ONLY RS 5000 WILL  0000503214113379   01/02/25                     5000      

##Normalize Column Names Using Fuzzy Matching

In [10]:
from fuzzywuzzy import process

def match_column(possible_names, actual_columns):
    match, score = process.extractOne(possible_names, actual_columns)
    return match if score > 60 else None

d:\codes\bank statement extractor\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


#train the model with expected data 

In [11]:
target_fields = {
    'date': ['date', 'transaction date', 'value date','Txn Date', 'Expense Date'],
    'description': ['description', 'particulars', 'payment name', 'narration'],
    'withdrawal': ['debit', 'withdrawal', 'amount withdrawn'],
    'deposit': ['credit', 'deposit', 'amount deposited']
}

In [12]:
#check what are the fields it is able to match

In [13]:
mapped_columns = {}

for field, options in target_fields.items():
    for option in options:
        match = match_column(option, df.columns)
        if match:
            mapped_columns[field] = match
            break

print(mapped_columns)

{'date': 'Date', 'description': 'Narration', 'withdrawal': 'Withdrawal Amt.', 'deposit': 'Deposit Amt.'}


# Extract and Clean the Data

In [ ]:
# Create empty dataframe to store cleaned data
df_clean = pd.DataFrame()

# Find first invalid date row
valid_mask = pd.to_datetime(df[mapped_columns['date']], errors='coerce').notna()
last_valid_idx = valid_mask[valid_mask].index[-1]

# Only take rows up to last valid date
df = df.iloc[:last_valid_idx + 1]

print(df.to_string())

In [ ]:
df_clean['Date'] = pd.to_datetime(df[mapped_columns['date']])
df_clean['Description'] = df[mapped_columns['description']].astype(str)
df_clean['Withdrawal'] = pd.to_numeric(df[mapped_columns['withdrawal']], errors='coerce').fillna(0)
df_clean['Deposit'] = pd.to_numeric(df[mapped_columns['deposit']], errors='coerce').fillna(0)
#coerce handles the errors and fillna handles the missing values
print(df_clean.head())

